# Lab 5 - Dynamic User Interfaces


So far, we have used **"static"** interfaces in our dashboards: 
the layout and the widgets were fixed; 
they were decided at the time of writing the program.
Now, we will start looking at how to **change the user interface itself dynamically**: 
the interface will change during **runtime**. 
We will see how we can change/update the widgets based on conditions that are controlled by the user, 
or how to add and remove new UI elements programmatically while the apps are running. 

**A dynamic user interface enhances the user experience in two ways:**
 - First, it provides a **responsive** interface: user gets visual feedback from the interaction with the interface. 
 - Second, it enables a cleaner and more efficient interface design: 
   - UI elements can be hidden until they are needed, 
   - less number of widgets can be made available at any given time, 
   - only relevant choices can be made visible depending on the current context (plot, data, etc.) 
 
The first dynamic interface example we will see is the ```conditionalPanel```.



## 1. Conditional Panels

When we use a ```sidebarLayout```, we display the widgets in the ```sidebarPanel``` for the user input. 
We can choose to display certain widgets based on user input using the **```conditionalPanel```**: 
it is *conditional*, so its display/visibility depends on the condition that is given as a *Javascript expression*. 
Do not worry about the Javascript part; 
it is basically a condition written within quotes, 
usually based on a widget's selection by the user. 
The following is a simple example to show the syntax:

---

```R
sidebarPanel(

    selectInput("type", "Type", c(Option1 = "op1", Option2 = "op2")),
    
    conditionalPanel(
                        condition = "input.type == 'op1'",
                        selectInput("select", "Something", c("S1", "S2", "S3"))
                    )
)

```


---

In this example, we have two widgets in the ```sidebarPanel```: one is always there;
other one's visibility depends on a condition. 
First widget is the selection list named ```"type"```; 
it has two options. Second widget is also a selection list named ```"select"```; 
it is given inside of a ```conditionalPanel```, so its visibility depends on the ```condition```. 

As said above, the ```condition``` is a Javascript expression that should be given inside double quotes. In this case, the expression is 

```
"input.type == 'op1'"
``` 

Because it is in Javascript and not R, we do **not** write it as 

```
"input$type == 'op1'" 

```

The condition simply implies that if the user selects ```op1``` in the **```type```** widget, the panel should be visible. In this case, the panel consists of only one widget that is the ```select``` selection list. We can add as many elements as we want after the condition to create a panel with multiple widgets. Also, we can **nest** conditional panels inside each other like this: 

---

```R
sidebarPanel(

    selectInput("type", "Type", c(Option1 = "op1", Option2 = "op2")),
    
    conditionalPanel(
                        condition = "input.type == 'op1'",
                        selectInput("select", "Something", c("S1", "S2", "S3")),
                        
                        conditionalPanel(
                                            condition = "input.select == 'S3'",
                                            
                                            # some other UI elements here
                                            selectInput(......)
                                        )   
                    )
)

```
---

In this example, second ```selectInput``` will be visible only if the user selects ```op1```, and the third element will be visible only if the user selects ```S3``` in the second ```selectInput```. 



### Example

Let's see a more useful example of this. We will use the **gapminder** data set, and will design a layout that lets the user to choose between a scatter plot and a bar chart based on a selection list. Here is the logic we will implement in the UI:

```
- Let user select between a scatter plot and a bar chart:
 - if scatter plot, let user choose the continent.
 - if bar chart, let user choose the year. 
```

For the first choice, we will use a ```selectInput``` selection list that will be always visible. Then, we will have **two consecutive** conditional panels:

 - First conditional panel will be visible in case of scatter plot is chosen.
 - Second conditional panel will be visible in case of bar chart is chosen. 

Note that these are **not nested** (unlike the example above). Inside these panels, we will use another ```selectInput``` to choose a continent, and a ```sliderInput``` to choose a year. 

**The following is the UI of the app**; study it carefully: 

As you can see above, there will be two widgets visible in the sidebarPanel at any time depending on the user input: 
either **```plotType```** and **```cont```** widgets, or **```plotType```** and **```year```** widgets.

Now, we need to implement the functionality in the server. 
Server code doesn't require anything special because of the ```conditionalPanel```; 
**we just make sure that we do not refer to an input variable that is not visible at the time.**

**Below is the server of the app.** 
Based on the value of the ```input$plotType```, 
we either display a scatter plot or a bar chart. 
We use the corresponding input variable:

 - in case of scatter plot, we use **```input$cont```** because that gives the chosen continent, 
 - in case of bar chart, we use **```input$year```** that comes from the slider control. 
 
 Note that this code is in R as always; 
 that's why the condition of the ```if``` statement below is an R expression, 
 do **not** mistake it for the Javascript expression of the **```condition```** in the UI code. 

In [ ]:
#DEPLOY TO SHINY SERVER
dir <- getwd() #This gets the current Working Directory
course <- "DATA-SCI-8654" #This is to specify the course path for the shiny server
folder <- "module5-lab1" #This specifies the folder name to copy

system(sprintf("/usr/local/bin/shiny_deploy %s %s %s", course, dir,folder), 
       intern = TRUE,
       ignore.stdout = FALSE, 
       ignore.stderr = FALSE,
       wait = TRUE, 
       input = NULL)

** Deploy and run this app**, and play with it to see how the second widget dynamically changes based on the selection in the first widget. 


Here, we can change the widget dynamically, but the choices are still fixed during the runtime of the app. Let's see how we can **change the choices** in a selection list dynamically.

## 2. Updating a Widget 

We can update the options of a widget during runtime. Let's see how we do it for a ```selectInput``` widget: 

```R
updateSelectInput(session, inputId, label = NULL, choices = NULL, selected = NULL)
```  
This is similar to the ```selectInput``` function with one difference: we need the **session** information. 

### Sessions: 

**When multiple users run the same Shiny app, how does the app know which user it is responding to?** All relevant information is stored in a **```session```** object. So far we did not need it, but with the dynamic interfaces, we will start using it. 

Here is a simple example of how to update a ```selectInput``` widget: 

---
```R
ui <- fluidPage(
  checkboxGroupInput("check", "Input checkbox", c("Item A", "Item B", "Item C")),
  selectInput("select", "Select input", c("Item A", "Item B", "Item C"))
)
```

 


---

This UI code does not really do anything; it just displays a checkbox and a selection list. The following server code dynamically changes the selection list: 

---
```R
server <- function(input, output, session) { # SESSION OBJECT IS PASSED HERE

  observe({

    x <- input$check # get a list of selections from the checkbox

    # we can use character(0) to remove all choices if nothing is checked yet. 
    if (is.null(x)) {    
      x <- character(0)
    }

    # we can set the label and selection list
    updateSelectInput(session, "select",
      label = paste("Select input label", length(x)), # update the label 
      choices = x,   # new list of choices 
      selected = tail(x, 1))  # pick one as the selected value 
  })
}

shinyApp(ui, server)
```
---


**There are a few things to note here:**

 - First, pay attention to the addition to the ```function``` arguments, now we get the **session** info from the server. 
 
 - Second, we use the **```observe()```** function to check if the user interacted with the widgets. It gets triggered every time ```check``` widget is changed.
 
 - Third, we use the **```updateSelectInput()```** function to change the selection list widget during runtime:
   - we make sure to use the ```session``` object passed to us, 
   - we use the name of the selection list widget we want to update (**```select```** here). 
 
 **Deploy and run this app** to see how it updates the selection list every time you check a box. 

In [ ]:
#DEPLOY TO SHINY SERVER
dir <- getwd() #This gets the current Working Directory
course <- "DATA-SCI-8654" #This is to specify the course path for the shiny server
folder <- "module5-lab2" #This specifies the folder name to copy

system(sprintf("/usr/local/bin/shiny_deploy %s %s %s", course, dir,folder), 
       intern = TRUE,
       ignore.stdout = FALSE, 
       ignore.stderr = FALSE,
       wait = TRUE, 
       input = NULL)

**We can create a list of choices from anything we want: we can get the levels of a column from a data frame based on the choices of the user, for example. **

Let's see how to update a few other widgets. 

### Updating the sliderInput

```sliderInput``` can be updated in the similar fashion: 

---
```R
updateSliderInput(session, inputId, label = NULL, value = NULL, min = NULL, max = NULL, step = NULL)

```
---

For example: 

---
```R
ui = fluidPage(
      sidebarLayout(
        sidebarPanel(
          p("The first slider controls the second"),
          sliderInput("control", "Controller:", min=0, max=20, value=10, step=1),
          sliderInput("receive", "Receiver:", min=0, max=20, value=10, step=1)
        ),
        mainPanel()
      )
)

server = function(input, output, session) {

      observe({
        val <- input$control

        # step size is 2 when input value is even; 1 when value is odd.
        updateSliderInput(session, "receive", 
                          value = val,
                          min = floor(val/2), max = val+4, step = (val+1)%%2 + 1)
      })
}

shinyApp(ui, server)
```
---


### Updating the other widgets 

Here is a short list of update functions that update the corresponding widgets; they follow similar syntax as their widget function counterparts with just the addition of the ```session``` variable. 

```R
updateRadioButtons(session, inputId, label = NULL, choices = NULL,
  selected = NULL, inline = FALSE, choiceNames = NULL,
  choiceValues = NULL)
  
updateActionButton(session, inputId, label = NULL, icon = NULL)
   
updateCheckboxInput(session, inputId, label = NULL, value = NULL)

updateCheckboxGroupInput(session, inputId, label = NULL, choices = NULL,
  selected = NULL, inline = FALSE, choiceNames = NULL,
  choiceValues = NULL)
```

  
## 3. Creating New UI Elements with renderUI 


In addition to updating the existing widgets, **we can also add new widgets or other UI elements to the app during runtime.** For that, we use the ```renderUI()``` function. Just like any other render function, it also needs an output function in the UI; that function is called ```uiOutput()```.

Here is a partial code example to show the idea: 

---

```R
# UI: 

ui <- fluidPage(
  # some other widgets here.... 
  uiOutput("somewidget") # this tells the UI that a widget will be rendered to this location. 
)

# server code: 

output$somewidget <- renderUI({ # render a widget to the output named somewidget 

  states <- levels(df$State)
  # this is the widget that will appear: it's a checkbox group
  checkboxGroupInput("check_state", "Choose States:", choices=states)
})

```
---

In the above example, we assume that there is a data frame ```df``` with a column named ```State```; we get a list of all states and make them the choices of a group of checkboxes which then we render to the ```uiOutput``` named ```"somewidget"```. We could create any widget we want, it doesn't have to be a checkbox group. The idea is that we have a *placeholder* in the UI for the widget we want to create during runtime. 

**How is this different from what we have done previously?** It gives us the ability to make a **widget appear whenever we want**, and also **replace it with another widget** if we want. Using an ```if``` statement, we could choose between a checkbox group and a slider, for example.

### ```insertUI``` and ```removeUI```

Instead of replacing a widget, **we can keep adding a new widget** or any UI element by using the ```insertUI``` function. 
Unlike ```renderUI```, the UI generated with ```insertUI``` is not updatable as a whole: once it’s created, it stays there. Each new call to ```insertUI``` creates more UI objects, in addition to the ones already there (all independent from one another). 
Similarly, ```removeUI``` removes an existing UI element. 

Below is an example to show how to do that. **Note that there is a ```tags``` element in the UI which merely serves as a place holder; we'll need that to tell the server code where to insert the new elements. **

In [ ]:
#DEPLOY TO SHINY SERVER
dir <- getwd() #This gets the current Working Directory
course <- "DATA-SCI-8654" #This is to specify the course path for the shiny server
folder <- "module5-lab3" #This specifies the folder name to copy

system(sprintf("/usr/local/bin/shiny_deploy %s %s %s", course, dir,folder), 
       intern = TRUE,
       ignore.stdout = FALSE, 
       ignore.stderr = FALSE,
       wait = TRUE, 
       input = NULL)

**Deploy and run this app** to familiarize with its functionality. The elements we are inserting in this example are simple HTML paragraphs with text:

```
tags$p("some text created on the fly")
```

but we could insert anything we want, such as widgets or more complicated HTML elements. Also, this code is for demonstration purposes only, so do not worry about how to maintain the ```inserted``` list; just try to understand the program logic. 

Here is a sample line of code that adds a ```sliderInput```:

```R
 insertUI(selector = '#placeholder', ui = sliderInput('slide1', 'Choose a value', 0, 100, 50)   )
```

and this is how to remove it: 

```R
 removeUI(selector = 'div:has(> #slide1)')
```   

Here, ```div:has()``` is a Javascript selector function that finds the element in the UI for us, and ```removeUI``` removes it. 

We can use insert and remove functions to add new widgets, plots, tables, or any HTML elements to the app during runtime. 